# Credit Card Fraud Detection By: Lokendra Pratap

## Import Dependencies

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Load Dataset

In [33]:
credit_card_data = pd.read_csv("creditcard.csv")

In [34]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [35]:
credit_card_data.shape

(284807, 31)

## Exploratory Data Analysis

In [36]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

#### Distribution of Transactions

In [37]:
credit_card_data["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [38]:
legit = credit_card_data[credit_card_data["Class"] == 0]
fraud = credit_card_data[credit_card_data["Class"] == 1]

#### Dataset is Highly unbalanced
0 -> Legit transaction
1 -> Fraud transaction

In [39]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


#### Statistical Analysis

In [40]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [41]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [42]:
credit_card_data.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Under Sampling
Build a sample Dataset of similar legit and Fraud Transactions i.e 492 Each

In [43]:
legit_sample = legit.sample(n = 492)

In [44]:
new_dataset = pd.concat([legit_sample,fraud], axis = 0)

In [45]:
new_dataset["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [46]:
new_dataset.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95316.276423,-0.032026,0.061317,-0.096996,0.116492,-0.029991,0.138807,0.023728,0.020018,0.040597,...,0.031925,0.055456,0.005147,0.022837,0.013829,-0.014783,-0.032583,0.003301,-0.001189,104.216585
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Features(X) and Targets(Y)

In [47]:
X = new_dataset.drop(columns = ["Class"], axis = 1)
Y = new_dataset.Class

In [48]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
92737,64071.0,1.465864,-1.151551,0.441787,-1.315312,-1.431418,-0.239680,-1.105070,0.005731,-1.683393,...,-0.333935,-0.370326,-0.572709,0.010478,-0.010257,0.416525,-0.266382,0.034761,0.006843,18.44
156220,107563.0,2.057305,-0.036035,-0.949585,0.624226,-0.150905,-1.112739,-0.010420,-0.316973,2.018276,...,-0.357547,-0.449107,-0.965729,0.374822,-0.109231,-0.379410,0.166836,-0.102463,-0.069808,0.89
182584,125467.0,1.976003,-0.469128,-0.752547,0.354374,-0.259679,-0.111036,-0.472534,-0.077815,1.016504,...,-0.030715,0.157032,0.521650,0.000648,-0.949646,-0.148294,0.411706,-0.020155,-0.044573,56.50
232325,147128.0,2.030720,-1.996815,-0.124240,-1.538481,-2.021411,-0.034834,-1.794754,0.147732,-0.788609,...,-0.249970,-0.051864,0.101042,0.226954,-0.383717,-0.609366,-0.262996,0.036804,-0.027720,109.72
268179,163097.0,1.648231,-0.831303,-3.248401,-0.512679,2.353871,3.282188,-0.285534,0.784295,0.859110,...,0.266605,-0.136584,-0.671010,0.062804,0.593820,-0.097576,-0.087442,-0.022990,0.004448,195.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [49]:
Y

92737     0
156220    0
182584    0
232325    0
268179    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

### Split into Training & Testing Data

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, stratify = Y, random_state = 2)

In [51]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
12415,21751.0,1.149144,-0.106709,1.001152,1.255606,-0.394607,1.000197,-0.986263,0.435282,2.088194,...,-0.304835,-0.026850,0.265404,-0.134795,-0.906488,0.426348,-0.210257,0.040967,0.002542,1.00
99486,67145.0,-1.046308,-0.358706,1.589153,0.626938,1.092017,-0.169739,-0.494536,0.494972,-0.551249,...,0.175059,0.026580,-0.277393,0.283729,-0.341478,-0.804951,0.214226,0.098688,0.145521,1.98
196010,131319.0,-0.571092,0.778269,-0.874651,-0.054922,0.935171,-1.450076,1.167339,-0.083609,-1.165323,...,0.108691,0.329717,0.722883,-0.058121,-0.075831,-0.434410,0.627218,-0.000633,0.066661,55.07
116404,74262.0,-2.250535,2.365755,-2.955491,0.089791,-2.830745,-0.844462,-0.174062,-0.407138,0.174216,...,-0.486537,0.256560,-0.466245,0.291105,0.242567,-1.279094,-1.123534,-0.630977,0.326839,311.28
240222,150494.0,1.852889,1.069593,-1.776101,4.617410,0.770413,-0.400859,-0.040970,0.089510,-0.217705,...,-0.288392,-0.157869,-0.176244,0.027437,-0.468006,0.058063,0.148263,0.042278,0.040573,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93191,64288.0,1.408156,-0.815645,0.258939,-1.198759,-0.437977,1.051761,-1.251603,0.340476,-0.520529,...,0.118665,0.346012,0.952883,-0.297762,-1.658671,0.549671,0.070113,0.046760,-0.005315,12.95
57470,47923.0,0.364377,1.443523,-2.220907,2.036985,-1.237055,-1.728161,-2.058582,0.358895,-1.393306,...,0.310980,0.402730,-0.132129,-0.032977,0.460861,0.560404,0.409366,0.539668,0.296918,0.76
41395,40662.0,-4.446847,-0.014793,-5.126307,6.945130,5.269255,-4.297177,-2.591242,0.342671,-3.880663,...,-0.108006,0.247913,-0.049586,-0.226017,-0.401236,0.856124,0.661272,0.492560,0.971834,1.00
230476,146344.0,-0.099724,2.795414,-6.423856,3.247513,-1.632290,-2.766665,-2.312223,0.961014,-1.896001,...,0.340898,0.647714,0.126576,0.203953,0.008495,-0.174501,0.575295,0.152876,-0.098173,94.82


In [52]:
Y_train

12415     0
99486     0
196010    0
116404    1
240222    1
         ..
93191     0
57470     1
41395     1
230476    1
254759    0
Name: Class, Length: 738, dtype: int64

In [53]:
X_test

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
151011,94364.0,-15.192064,10.432528,-19.629515,8.046075,-12.838167,-1.875859,-21.359738,-3.717850,-5.969782,...,1.657476,-3.474097,1.765446,1.701257,0.381587,-1.413417,-1.023078,-2.634761,-0.463931,1.00
116206,74186.0,-2.313276,2.674914,0.207198,0.090650,-1.000285,0.587370,-3.100966,-7.409354,-1.344694,...,1.885282,-3.949034,0.608037,0.704331,0.301483,-0.373610,0.023422,0.037131,0.165487,9.99
267674,162870.0,-0.192549,1.300759,-1.254141,-0.529186,0.892557,-1.176471,1.118232,0.109285,-0.787749,...,-0.123572,0.318851,0.998630,-0.103483,-0.305648,-0.697564,-0.293221,0.358736,0.241749,5.90
12890,22644.0,-0.518531,1.062806,2.053490,3.150431,-1.129058,0.596819,1.809551,-0.590833,-0.155819,...,0.243920,-0.261910,-0.267033,0.369732,0.646018,-0.755771,-0.184249,-0.010972,-0.136224,301.82
42756,41233.0,-10.645800,5.918307,-11.671043,8.807369,-7.975501,-3.586806,-13.616797,6.428169,-7.368451,...,-0.046170,2.571970,0.206809,-1.667801,0.558419,-0.027898,0.354254,0.273329,-0.152908,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59320,48790.0,1.246367,-0.818360,-0.615706,-0.678617,-0.251785,-0.187017,0.001629,-0.305584,-0.890780,...,-0.109303,-0.447125,-0.921864,-0.309404,-0.919183,0.609885,1.160227,-0.088509,0.009632,140.79
42528,41138.0,-4.595617,5.083690,-7.581015,7.546033,-6.949165,-1.729185,-8.190192,2.714670,-7.083169,...,1.682160,2.248971,0.566844,0.033744,0.591783,0.334229,0.386801,2.163898,0.983104,340.11
112840,72824.0,-1.111495,-0.257575,2.250210,1.152671,0.432904,1.254126,-0.584163,-0.609682,1.014602,...,-0.510614,0.862913,0.927825,-0.343058,-0.256268,-0.600742,-0.180331,0.026762,-0.358335,45.03
213035,139087.0,1.922986,0.450722,-0.660450,3.841270,0.224751,-0.279812,0.119796,-0.111185,-0.702324,...,-0.342648,0.275074,0.856979,-0.005693,-0.034883,0.193037,0.248302,-0.040273,-0.054002,10.62


In [54]:
Y_test

151011    1
116206    0
267674    0
12890     0
42756     1
         ..
59320     0
42528     1
112840    1
213035    0
2072      0
Name: Class, Length: 246, dtype: int64

## Model Training
Logistic Regression because Binary Classification

In [55]:
model = LogisticRegression()

In [56]:
model = model.fit(X_train, Y_train)

c:\Users\Lokendra Pratap\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Evaluation
### Accuracy Score

In [57]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

0.9417344173441734

In [58]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.9186991869918699

## Deploy
### Creating Pickle File

In [59]:
import pickle

In [60]:
pickle.dump(model, open("model.pkl", 'wb'))

### Creating Flask API

In [61]:
import numpy as np
from flask import Flask, request, jsonify, render_template

In [62]:
app = Flask(__name__)
calc = pickle.load(open("model.pkl", 'rb'))